In [14]:
import pandas as pd 
import awoc
import numpy as np
import re

In [15]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [16]:
df = pd.read_csv("monumentos_world.csv", index_col=0)

In [17]:
df.head(2)

,Imagen,N.º,PH,Nombre,Ciudad,País,Visitantes/año,Año,#
0,Shwedagon Pagoda 2017.jpg,f/o,—,Gran Pagoda de Shwedagon,Yangon,Bandera de Birmania Birmania,615 763,201747​,—
1,Kerbela Hussein Moschee.jpg,—,—,Tumba del Imán Hussein,Kerbala,Bandera de Irak Irak,45 000 000,?,—


____________

### 1. Empezamos con un EDA básico

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206 entries, 0 to 205
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Imagen          206 non-null    object
 1   N.º             204 non-null    object
 2   PH              205 non-null    object
 3   Nombre          204 non-null    object
 4   Ciudad          159 non-null    object
 5   País            159 non-null    object
 6   Visitantes/año  112 non-null    object
 7   Año             113 non-null    object
 8   #               109 non-null    object
dtypes: object(9)
memory usage: 16.1+ KB


In [19]:
print(f'La cantidad total de columnas es', df.shape[0], 'columnas y', df.shape[1], 'filas.')


La cantidad total de columnas es 206 columnas y 9 filas.


In [20]:
df.columns

Index(['Imagen', 'N.º', 'PH', 'Nombre', 'Ciudad', 'País', 'Visitantes/año',
       'Año', '#'],
      dtype='object')

Cambiamos el nombre de las columnas para que no tenga mayúsculas ni acentos

In [21]:
lista_cols = [x.replace("í", "i").replace("/", "_").lower() for x in df.columns]

In [22]:
# añadimos de manera "manual" a la columna de año "_conteo", para poder diferenciarlo más 
# adelante de la futura columna de "año_monumento", por ejemplo

lista_cols = ['imagen','n.º','ph','nombre','ciudad','pais','visitantes_año','año_conteo','#']
df.columns = lista_cols

In [23]:
print(df.duplicated().sum())
print('------------------------')
print(df.isnull().sum() *100 /df.shape[0])

0
------------------------
imagen             0.000000
n.º                0.970874
ph                 0.485437
nombre             0.970874
ciudad            22.815534
pais              22.815534
visitantes_año    45.631068
año_conteo        45.145631
#                 47.087379
dtype: float64


In [24]:
df.drop(["imagen","ph", "n.º", "#", ], axis=1, inplace=True)

In [25]:
df.head(1)

,nombre,ciudad,pais,visitantes_año,año_conteo
0,Gran Pagoda de Shwedagon,Yangon,Bandera de Birmania Birmania,615 763,201747​


_________

### 2. Sacamos la columna de continente

1º "limpiamos" la columna de paises

In [26]:
df["pais"] = df["pais"].str.split()

In [27]:
def pais(col):
    try:
        col = col[-1]
        return col
    except:
        pass
df.head(2)

,nombre,ciudad,pais,visitantes_año,año_conteo
0,Gran Pagoda de Shwedagon,Yangon,"[Bandera, de, Birmania, Birmania]",615 763,201747​
1,Tumba del Imán Hussein,Kerbala,"[Bandera, de, Irak, Irak]",45 000 000,?


In [28]:
#Lo almacenamos en una columna nueva por seguir manteniendo la información original
df["pais_corto"] = df["pais"].apply(pais) 
df.head(2)

,nombre,ciudad,pais,visitantes_año,año_conteo,pais_corto
0,Gran Pagoda de Shwedagon,Yangon,"[Bandera, de, Birmania, Birmania]",615 763,201747​,Birmania
1,Tumba del Imán Hussein,Kerbala,"[Bandera, de, Irak, Irak]",45 000 000,?,Irak


In [29]:
df.pais_corto.unique()

array(['Birmania', 'Irak', 'India', 'Irán', 'Japón', None, '—', 'Brasil',
       'México', 'Francia', '01', 'Portugal', 'Unidos', 'China', 'Italia',
       '03', '858', 'Unido', 'Rusia', 'España', 'Polonia', '20', 'Sur',
       '32', 'Turquía', 'Australia', '406', '300', 'Bajos', 'Israel',
       '63', 'Alemania', 'Austria'], dtype=object)

2º De la columna "limpia de paises" sacamos la columna nueva de continente

In [30]:
my_world = awoc.AWOC()

In [31]:
#Creamos las listas por Continentes de los paises a los que hace referencia.
def continente(col):
    asia = ["Arabia Saudita","Birmania" ,"Camboya",'China',"Corea del Sur", "Emiratos Árabes", "India", "Irak", "Irán",'Israel',"Japón", "Taiwan","Tokio","Turquía"]
    europa =  ['Alemania','Austria','Italia','España','Estambul','Francia','Paises Bajos','Polonia','Portugal', 'Reino Unido','Rusia', 'Venecia','Ámsterdam','Edimburgo', 'Milán', 'Grecia', 'Ciudad del Vaticano', 'Nápoles', 'Varsovia']
    america = ["Estados Unidos", "Brasil", "México", "Unidos", 'Ciudad de México',"Puerto Rico", "Perú"]
    oceania =['Australia']
    africa = ['Egipto']

    try:
        if col in asia:
            return "Asia"
        elif col in europa:
            return "Europa"
        elif col in america:
            return "America"
        elif col in africa:
            return "Africa"
        else:
            return my_world.get_country_continent_name(col)
    except:
        pass

In [32]:
df["continente"] = df["pais_corto"].apply(continente)

In [33]:
df["continente"].unique()

array(['Asia', None, 'America', 'Europa', 'Oceania'], dtype=object)

_____________

### 3. "Rellenamos" la columna de ciudad con los datos correspondientes

In [34]:
df.ciudad.unique()

array(['Yangon', 'Kerbala', 'Kerala', 'Mashad', 'Tokio', 'Benarés',
       'Andhra Pradesh', nan, '20182\u200b', '?3\u200b', 'Aparecida',
       '20184\u200b5\u200b', 'Ciudad de México', 'París',
       'Prefectura de Chiba', 'Kamakura', '20121\u200b', '201921\u200b',
       'Ourém', 'Ise', '20188\u200b', 'Washington D. C.', '201810\u200b',
       'Beijing', '201611\u200b', '2018 12\u200b', '201713\u200b', 'Roma',
       'Flag of the United Arab Emirates.svg Emiratos Árabes Unidos',
       'Dazaifu', 'San Giovanni Rotondo', 'Nueva York', 'Londres',
       'Lourdes', '201815\u200b', '201616\u200b', 'Venecia', 'Saitama',
       'San Petersburgo', 'Filadelfia', 'Barcelona', 'Nueva Delhi',
       '?7\u200b', 'Madrid', 'Czestochowa', 'Florencia', 'Hangzhou',
       'Washington, D.C.', '201820\u200b', 'Taipéi', '201922\u200b',
       'Nanjing', '2018 23\u200b', '2018', 'Changsha', 'Shanghái',
       'Gansu', 'Seúl', '[[]]', 'Varsovia', '201725\u200b',
       '201726\u200b', 'Taichung', '2018

In [35]:
# df[df["ciudad"]== '20182\u200b']
# df.iloc[8,2] = "Pekin"
# df.iloc[8,2]

  > Identificamos por medio de los indices, los valores alfanumericos de la columna 'ciudad' y los sustituimos por los nombres de las ciudades correspondientes.

In [36]:
df.iloc[7,1] = "Pekín"
df.iloc[9,1] = 'París'
df.iloc[12,1] = 'Ciudad del Vaticano'
df.iloc[18,1] = 'Kioto'
df.iloc[20,1] = 'París'
df.iloc[23,1] = 'China'
df.iloc[26,1] = 'Versalles'
df.iloc[29,1] = 'Roma'
df.iloc[31,1] = 'Sidney'
df.iloc[34,1] = 'Atenas'
df.iloc[36,1] = 'Agra'
df.iloc[38,1] = 'París'
df.iloc[41,1] = 'Ciudad del Vaticano'
df.iloc[51,1] = 'Kioto'
df.iloc[53,1] = 'Colonia'
df.iloc[56,1] = 'Cracovia'
df.iloc[59,1] = 'San Petesburgo'
df.iloc[61,1] = 'Estambul'
df.iloc[64,1] = 'Kioto'
df.iloc[73,1] = 'Ciudad del Vaticano'
df.iloc[75,1] = 'Florencia'
df.iloc[86,1] = 'Teotihuacán'
df.iloc[90,1] = 'Estambul'
df.iloc[93,1] = 'San Petesburgo'
df.iloc[95,1] = 'París'
df.iloc[104,1] = 'San Petesburgo'
df.iloc[109,1] = 'Hong Kong'
df.iloc[114,1] = 'Viena'
df.iloc[116,1] = 'Guiza'
df.iloc[117,1] = 'Viena'
df.iloc[119,1] = 'Kazan'
df.iloc[122,1] = 'Moscu'
df.iloc[124,1] = 'Londres'
df.iloc[133,1] = 'Granada'
df.iloc[135,1] = 'México'
df.iloc[141,1] = 'Camboya'
df.iloc[146,1] = 'Pamukkale'
df.iloc[174,1] = 'Auschwitz'
df.iloc[179,1] = 'Córdoba'
df.iloc[182,1] = 'Sevilla'
df.iloc[188,1] = 'Amesbury'
df.iloc[190,1] = 'Londres'
df.iloc[195,1] = 'Cuzco' 
df.iloc[196,1] = 'Puerto Rico'
df.iloc[199,1] = 'Haifa'
df.iloc[203,1] = 'Moscú'

In [68]:
df.sample(5)

,nombre,ciudad,pais,visitantes_año,año_conteo,pais_corto,continente
94,Nanjing Museum,Nanjing,"[Bandera, de, la, República, Popular, China, C...",3 670 000,201838​,China,Asia
141,Angkor Wat,Camboya,NaN,NaN,NaN,None,None
119,Kremlin de Kazán,Kazan,NaN,NaN,NaN,None,None
140,Estatua de la Unidad,Kevadiya (Guyarat),"[Bandera, de, la, India, India]",2 600 000+,201933​,India,Asia
70,Parque nacional de la Independencia,Filadelfia,"[Bandera, de, Estados, Unidos, Estados, Unidos]",4 576 436,20189​,Unidos,America


> Eliminamos las columnas con informacion redundante y reseteamos indices. Corresponden a las lineas: 117, 142, 174, 182, 188, 195. Guardamos en un df(df1) nuevo tras borrar las columnas. 

In [38]:
df1= df.drop([8,10,13,19,21,24,27,30,32,35,37,39,42,52,54,57,60,62,65,74,76,87,91,92,93,96,97,98,100,105,115,120,123,125,
        134,136,142,147,180,181,187,191,194,197,200,204], axis=0, inplace=True)
df1= df.reset_index(drop=True)

In [39]:
print(f'Tras eliminar las filas con infromacion redundante de nuestro df, tenemos un total de ', 
        df1.shape[0], 'filas, y de ', df1.shape[1], 'columnas.')

Tras eliminar las filas con infromacion redundante de nuestro df, tenemos un total de  160 filas, y de  7 columnas.


In [40]:
df1.head(162)

,nombre,ciudad,pais,visitantes_año,año_conteo,pais_corto,continente
0,Gran Pagoda de Shwedagon,Yangon,"[Bandera, de, Birmania, Birmania]",615 763,201747​,Birmania,Asia
1,Tumba del Imán Hussein,Kerbala,"[Bandera, de, Irak, Irak]",45 000 000,?,Irak,Asia
2,Templo de Sabarimala,Kerala,"[Bandera, de, la, India, India]",40 000 000,?,India,Asia
3,Tumba del Gran Imán Reza,Mashad,"[Bandera, de, Irán, Irán]",30 000 000,?,Irán,Asia
4,Santuario Meiji,Tokio,"[Bandera, de, Japón, Japón]",30 000 000,20121​,Japón,Asia
5,Templo de Kashi Vishwanath,Benarés,"[Bandera, de, la, India, India]",21 900 000,20121​,India,Asia
6,Templo de Tirupati Tirumala Devasthanams,Andhra Pradesh,"[Bandera, de, la, India, India]",18 250 000,20121​,India,Asia
7,Ciudad Prohibida,Pekín,NaN,NaN,NaN,None,None
8,Catedral de Notre Dame,París,NaN,NaN,NaN,None,None
9,Basílica de Nuestra Señora Aparecida,Aparecida,"[Bandera, de, Brasil, Brasil]",13 000 000,20176​,Brasil,America


> Modificamos los valores numericos de la columna 'nombre' por el momumento correspondiente. 

In [41]:
dicc_nombre_monumentos={'3 050 000':['Palacio de Schönbrunn'], '2 100 000':['Museo estatal Auschwitz'], 
                        '1 875 744':['Real Alcázar de Sevilla'], '1 555 868':['Stoneheng'], 
                        '1 411 276': ['Machu Pichu']}

df1['nombre']=df1['nombre'].replace(dicc_nombre_monumentos)

In [42]:
df1.head(165)

,nombre,ciudad,pais,visitantes_año,año_conteo,pais_corto,continente
0,Gran Pagoda de Shwedagon,Yangon,"[Bandera, de, Birmania, Birmania]",615 763,201747​,Birmania,Asia
1,Tumba del Imán Hussein,Kerbala,"[Bandera, de, Irak, Irak]",45 000 000,?,Irak,Asia
2,Templo de Sabarimala,Kerala,"[Bandera, de, la, India, India]",40 000 000,?,India,Asia
3,Tumba del Gran Imán Reza,Mashad,"[Bandera, de, Irán, Irán]",30 000 000,?,Irán,Asia
4,Santuario Meiji,Tokio,"[Bandera, de, Japón, Japón]",30 000 000,20121​,Japón,Asia
5,Templo de Kashi Vishwanath,Benarés,"[Bandera, de, la, India, India]",21 900 000,20121​,India,Asia
6,Templo de Tirupati Tirumala Devasthanams,Andhra Pradesh,"[Bandera, de, la, India, India]",18 250 000,20121​,India,Asia
7,Ciudad Prohibida,Pekín,NaN,NaN,NaN,None,None
8,Catedral de Notre Dame,París,NaN,NaN,NaN,None,None
9,Basílica de Nuestra Señora Aparecida,Aparecida,"[Bandera, de, Brasil, Brasil]",13 000 000,20176​,Brasil,America


_______________________

### 4. Limpiamos la columna de año

In [43]:
df1.año_conteo.unique() # Vemos que el año cuando se tomaron el conteo de gente son los primeros 4 numeros, como no sabemos a qué hace referencia los siguientes los quitaremos.

array(['201747\u200b', '?', '20121\u200b', nan, '20176\u200b', '?7\u200b',
       '2018 9\u200b', '201921\u200b', '—', '201949\u200b', '20189\u200b',
       '201817\u200b', '2018 18\u200b', '201850\u200b', '201919\u200b',
       '201838\u200b', '2019 51\u200b', '201952\u200b', '201827\u200b',
       '201753\u200b', '201624\u200b', '201854\u200b', '201855\u200b',
       '201831\u200b', '201832\u200b', '201933\u200b', '201856\u200b',
       '2017 57\u200b', '201858\u200b', '201859\u200b', '201836\u200b',
       '201760\u200b', '201837\u200b', '201739\u200b', '201761\u200b',
       '201842\u200b', '2017', '201843\u200b', '201744\u200b'],
      dtype=object)

In [44]:
def limpia_año(col):
    patron = r"\d{4}"
    try:
        return re.findall(patron, col)[0]
    except:
        pass

In [45]:
df1["año_conteo"] = df1["año_conteo"].apply(limpia_año)
df1.head(2)

,nombre,ciudad,pais,visitantes_año,año_conteo,pais_corto,continente
0,Gran Pagoda de Shwedagon,Yangon,"[Bandera, de, Birmania, Birmania]",615 763,2017,Birmania,Asia
1,Tumba del Imán Hussein,Kerbala,"[Bandera, de, Irak, Irak]",45 000 000,None,Irak,Asia


In [46]:
df1.año_conteo.unique()

array(['2017', None, '2012', '2018', '2019', '2016'], dtype=object)

Convertimos a float y no a int porque la columna tiene NaNs

In [47]:
df1["año_conteo"] = df1["año_conteo"].astype(float)

____________________

### 5. "Limpiamos" y cambiamos el tipo de dato de la columna "visitantes_año"

In [48]:
df1["visitantes_año"].unique()

array(['615 763', '45 000 000', '40 000 000', '30 000 000', '21 900 000',
       '18 250 000', nan, '13 000 000', '10 600 000', '10 500 000',
       '10 000 000', '9 400 000', '8 500 000', '7 804 683', '7 390 000',
       '7 000 000', '2019 14\u200b', '6 600 000', '6 479 548',
       '6 239 983', '6 098 340', '6 011 007', '6 000 000', '5 500 000',
       '5 300 000', '5 000 000', '4 956 529', '4 719 148', '4 652 865',
       '4 576 436', '4 500 000', '4 425 699', '4 400 000', '4 391 861',
       '4 335 431', '4 200 000', '4 074 403', '3 932 938', '3 832 373',
       '3 670 000', '3 600 000', '3 540 000', '3 500 000', '3 354 161',
       '3 286 000', '3 273 867', '3 232 730', '3 200 000', '3 143 626',
       '3 100 000', '3 098 985', '2 963 857', '2 893 000', '2 830 000',
       '2 817 386', '2 800 000', '2 787 770', '2 780 000', '2 774 103',
       '2 717 565', '2 661 615', '2 637 567', '2 600 000+', '2 596 725',
       '2 565 474', '2 560 000', '2 550 000', '2 549 833', '2 520 000',
 

1º Eliminamos los valores corruptos

In [49]:
lista = [ "2019 14\u200b", "201935\u200b", "201616\u200b", "+" ] #, " "]

for x in lista:
    df1["visitantes_año"] = df1["visitantes_año"].str.replace(x, "")

/var/folders/f0/204ksvss62b9dwtcgg8q17540000gn/T/ipykernel_10513/2900614661.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df1["visitantes_año"] = df1["visitantes_año"].str.replace(x, "")


In [50]:
df1["visitantes_año"].unique()

array(['615 763', '45 000 000', '40 000 000', '30 000 000', '21 900 000',
       '18 250 000', nan, '13 000 000', '10 600 000', '10 500 000',
       '10 000 000', '9 400 000', '8 500 000', '7 804 683', '7 390 000',
       '7 000 000', '', '6 600 000', '6 479 548', '6 239 983',
       '6 098 340', '6 011 007', '6 000 000', '5 500 000', '5 300 000',
       '5 000 000', '4 956 529', '4 719 148', '4 652 865', '4 576 436',
       '4 500 000', '4 425 699', '4 400 000', '4 391 861', '4 335 431',
       '4 200 000', '4 074 403', '3 932 938', '3 832 373', '3 670 000',
       '3 600 000', '3 540 000', '3 500 000', '3 354 161', '3 286 000',
       '3 273 867', '3 232 730', '3 200 000', '3 143 626', '3 100 000',
       '3 098 985', '2 963 857', '2 893 000', '2 830 000', '2 817 386',
       '2 800 000', '2 787 770', '2 780 000', '2 774 103', '2 717 565',
       '2 661 615', '2 637 567', '2 600 000', '2 596 725', '2 565 474',
       '2 560 000', '2 550 000', '2 549 833', '2 520 000', '2 460 000',
  

2º convertimos a número

In [51]:
#df["visitantes_año"] = pd.to_numeric(df["visitantes_año"])   
# Dejo comentada la fila porque si convierto a float luego no puedo filtrar por value_counts

______

### 6. Años anteriores a 2018

In [52]:
df1["año_conteo"].unique()

array([2017.,   nan, 2012., 2018., 2019., 2016.])

In [53]:
lista_años = [2012 ,2016 , 2017]

df_anteriores = df1[df1["año_conteo"].isin(lista_años) ]

In [54]:
#Ordenamos los indices
df_anteriores.sort_index()

,nombre,ciudad,pais,visitantes_año,año_conteo,pais_corto,continente
0,Gran Pagoda de Shwedagon,Yangon,"[Bandera, de, Birmania, Birmania]",615 763,2017.0,Birmania,Asia
4,Santuario Meiji,Tokio,"[Bandera, de, Japón, Japón]",30 000 000,2012.0,Japón,Asia
5,Templo de Kashi Vishwanath,Benarés,"[Bandera, de, la, India, India]",21 900 000,2012.0,India,Asia
6,Templo de Tirupati Tirumala Devasthanams,Andhra Pradesh,"[Bandera, de, la, India, India]",18 250 000,2012.0,India,Asia
9,Basílica de Nuestra Señora Aparecida,Aparecida,"[Bandera, de, Brasil, Brasil]",13 000 000,2017.0,Brasil,America
13,Templo Naritasan Shinshoji,Prefectura de Chiba,"[Bandera, de, Japón, Japón]",10 000 000,2012.0,Japón,Asia
14,Tsurugaoka Hachiman-gū,Kamakura,"[Bandera, de, Japón, Japón]",10 000 000,2012.0,Japón,Asia
19,Santuario de Ise,Ise,"[Bandera, de, Japón, Japón]",8 500 000,2012.0,Japón,Asia
31,Dazaifu Tenman-gū,Dazaifu,"[Bandera, de, Japón, Japón]",6 600 000,2012.0,Japón,Asia
32,Santuario del Padre Pío en San Giovanni Rotondo,San Giovanni Rotondo,"[Flag, of, Italy.svg, Italia]",6 600 000,2012.0,Italia,Europa


In [55]:
df_anteriores.shape

(31, 7)

In [56]:
df1.drop(['pais'], axis=1, inplace=True)

In [57]:
df1.head(2)

,nombre,ciudad,visitantes_año,año_conteo,pais_corto,continente
0,Gran Pagoda de Shwedagon,Yangon,615 763,2017.0,Birmania,Asia
1,Tumba del Imán Hussein,Kerbala,45 000 000,NaN,Irak,Asia


In [58]:
#Modificamos el nombre de 
df1.rename({'pais_corto':'pais'}, axis=1)
df.head(2)

,nombre,ciudad,pais,visitantes_año,año_conteo,pais_corto,continente
0,Gran Pagoda de Shwedagon,Yangon,"[Bandera, de, Birmania, Birmania]",615 763,201747​,Birmania,Asia
1,Tumba del Imán Hussein,Kerbala,"[Bandera, de, Irak, Irak]",45 000 000,?,Irak,Asia


31 valores a actualizar en la columna de "visitantes_año"

__________

### 6.2 años posteriores a 2018

In [59]:
lista_años_2 = [2018, 2019]

df_posteriores = df1[df1["año_conteo"].isin(lista_años_2) ]

In [60]:
df_posteriores.shape

(69, 6)

________________

### 6.3 Sacar valores nulos

Tenemos 106 valores nulos en la columna "visitantes_año". porque en el filtro de anteriores y posteriores tenemos un total de 100 y el total de las filas es de 206. Lo dejo como pendientes de sacar.

_______________

### Otros filtros

In [61]:
df_asia = df1[df1["continente"]== "Asia"]
df_america = df1[df1["continente"]== "America"]
df_europa = df1[df1["continente"]== "Europa"]
df_africa = df1[df1["continente"]== "Africa"]
df_oceania = df1[df1["continente"]== "Oceania"]

In [62]:
# Posible "filtro" para quitar valores nans



# lista_visitantes = ['615 763', '45 000 000', '40 000 000', '30 000 000', '21 900 000',
#        '18 250 000', '13 000 000', '10 600 000', '10 500 000',
#        '10 000 000', '9 400 000', '8 500 000', '7 804 683', '7 390 000',
#        '7 000 000',  '6 600 000', '6 479 548',
#        '6 239 983', '6 098 340', '6 011 007', '6 000 000', '5 500 000',
#        '5 300 000', '5 000 000', '4 956 529', '4 719 148', '4 652 865',
#        '4 576 436', '4 500 000', '4 425 699', '4 400 000', '4 391 861',
#        '4 335 431', '4 200 000', '4 074 403', '3 932 938', '3 832 373',
#        '3 670 000', '3 600 000', '3 540 000', '3 500 000', '3 354 161',
#        '3 286 000', '3 273 867', '3 232 730', '3 200 000', '3 143 626',
#        '3 100 000', '3 098 985', '2 963 857', '2 893 000', '2 830 000',
#        '2 817 386', '2 800 000', '2 787 770', '2 780 000', '2 774 103',
#        '2 717 565', '2 661 615', '2 637 567', '2 600 000+', '2 596 725',
#        '2 565 474', '2 560 000', '2 550 000', '2 549 833', '2 520 000',
#         '2 460 000', '2 431 073', '2 400 000',
#         '2 340 000', '2 313 000', '2 304 404', '2 300 000',
#        '2 250 000', '2 240 000', '2 231 000', '2 227 773', '2 192 200',
#        '2 190 000', '2 165 601', '2 161 160', '2 148 538', '2 111 730',
#        '2 111 578', '2 100 000', '2 054 719', '2 000 000', '1 981 000',
#        '1 700 000', '1 657 446', '1 583 260', '1 552 481', '1 500 000',
#        '1 300 000', '1 000 000+', '1 000 000']
# 
# df_2 = df[df["visitantes_año"].isin(lista_visitantes)]

In [63]:
# Esta celda y la siguiente para filtrar por los top 10 de visitas según contiente. No aplicar hasta decidir como rellenar los datos valores no actualizados o nans de la columna "visitantes_año"

# df_europa["visitantes_año"].value_counts()[:10]

In [64]:
# lista_europa = [ "10 500 000" ,"2 161 160","2 637 567" ,"2 549 833" ,"2 300 000" ,"2 231 000" ,"2 227 773" ,"2 192 200" ,"2 165 601" ,"2 148 538"  ]
# df_europa_2 = df_europa[df_europa["visitantes_año"].isin(lista_europa)]
# df_europa_2



# df_total = pd.concat([df_europa_2,df_america_2,df_asia_2,df_oceania, df_africa], axis=0).reset_index()